<a href="https://colab.research.google.com/github/eddyxu/rikai/blob/lei%2Funcertainty_sampling/notebooks/UncertaintySampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Uncertainty Sampling

[Active Learning](https://en.wikipedia.org/wiki/Active_learning_(machine_learning)) is one such technique that helps ML practitioners intelligently select high-value raw data to be labeled to maximize model improvement. Studies have shown that this type of learning strategy can produce significant improvement in model accuracy for the same number of samples; or require significantly fewer examples to reach the same accuracy.


In [1]:
!python -V
!nvidia-smi
!java --version

Python 3.7.12
Sat Jan 29 06:48:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                          

In [2]:
!pip install -q --force -U git+https://github.com/eddyxu/rikai.git@lei/uncertainty_sampling#egg=rikai\&subdirectory=python
!pip install -q mlflow==1.22 torch torchvision

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.3.post1 requires numpy<1.20,>=1.16.0, but you have numpy 1.21.5 which is incompatible.
nbclient 0.5.10 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
moviepy 0.2.3.5 requires decorator<5.0,>=4.0.2, but you have decorator 5.1.1 which is incompatible.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.26 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.31.1 which is incompatible.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.5 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
google-colab 1.0.0 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
da

In [3]:
import mlflow
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from rikai.spark.utils import get_default_jar_version

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

rikai_version = get_default_jar_version(use_snapshot=False)

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
spark = (
    SparkSession
    .builder
    .config("spark.jars.packages", f"ai.eto:rikai_2.12:0.1.0")
    .config(
        "spark.sql.extensions",
        "ai.eto.rikai.sql.spark.RikaiSparkSessionExtensions",
    )
    .config(
        "spark.rikai.sql.ml.registry.mlflow.tracking_uri",
        MLFLOW_TRACKING_URI,
    )
    .config("spark.executor.memory", "4g")
    .config("spark.driver.memory", "1g")
    .master("local[2]")
    .getOrCreate()
);

In [4]:
# From https://pytorch.org/vision/0.11/models.html#object-detection-instance-segmentation-and-person-keypoint-detection

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

@udf
def coco_name(idx: int) -> str:
  return COCO_INSTANCE_CATEGORY_NAMES[idx]

spark.udf.register("coco_name", coco_name)

# Create a category table
spark.createDataFrame(
    [{"label": idx, "name": name} for idx, name in enumerate(COCO_INSTANCE_CATEGORY_NAMES)]
).createOrReplaceTempView("cats")

# Preparing data

We use Coco's evaluation dataset [^1] to prepare a [Rikai](https://github.com/eto-ai/rikai) dataset. 

Note that, we only use eval set because it can fit into Google Colab's disk quota. 


In [5]:
from pathlib import Path

coco_dir = Path("./coco")
if not coco_dir.exists() and True:
  !mkdir -p coco
  !wget http://images.cocodataset.org/zips/val2017.zip -O coco/val2017.zip
  !wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip -O coco/annotations.zip
  !cd coco && find . -name '*.zip' -exec unzip -qq {} \; && rm *.zip

from rikai.contrib.datasets.coco import convert_instance

!rm -rf spark-warehouse/
df = convert_instance(
    spark, 
    "coco/annotations/instances_val2017.json",
    "coco/val2017",
)
df.write.mode("overwrite").save("datasets/coco")
df.createOrReplaceTempView("coco")

# Lets also create a COCO-1000 dataset for faster purpose purpose
# on Google CoLab.
df.limit(1000).createOrReplaceTempView("coco_1k")

In [6]:
spark.sql("SHOW TABLES").show()
spark.sql("SELECT COUNT(*) FROM coco").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|        |     cats|       true|
|        |     coco|       true|
|        |  coco_1k|       true|
+--------+---------+-----------+

+--------+
|count(1)|
+--------+
|    4952|
+--------+



In [7]:
spark.sql("SELECT * FROM coco").printSchema()


root
 |-- date_captured: string (nullable = true)
 |-- width: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- file_name: string (nullable = true)
 |-- image: image (nullable = true)
 |-- image_id: integer (nullable = true)
 |-- annotations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- image_id: integer (nullable = true)
 |    |    |-- area: double (nullable = true)
 |    |    |-- label_id: integer (nullable = true)
 |    |    |-- ann_id: long (nullable = true)
 |    |    |-- bbox: box2d (nullable = true)
 |    |    |-- segmentation: mask (nullable = true)
 |    |    |-- supercategory: string (nullable = true)
 |    |    |-- label: string (nullable = true)



# Register Models

We use a pre-trained [PyTorch SSD object detection model](https://pytorch.org/vision/0.11/models.html#object-detection-instance-segmentation-and-person-keypoint-detection). Rikai also comes with a customized model (`SSDClassScoresExtractor`) that uses SSD to extract detection classes and scores. 

We use [Mlflow](https://mlflow.org/) as a model registry to serve models into Rikai SQL-ML engine. Once the registration is done, you can list the models in the Rikai model catalog:

In [8]:
import rikai
from torchvision.models.detection.ssd import ssd300_vgg16
from rikai.contrib.torch.inspect.ssd import SSDClassScoresExtractor
from rikai.contrib.torch.detections import OUTPUT_SCHEMA

ssd = ssd300_vgg16(pretrained=True)
class_scores_extractor = SSDClassScoresExtractor(ssd, topk_candidates=90)

print(OUTPUT_SCHEMA)

with mlflow.start_run():
    rikai.mlflow.pytorch.log_model(
        ssd, 
        "model", 
        OUTPUT_SCHEMA,
        pre_processing="rikai.contrib.torch.transforms.ssd.pre_processing",
        post_processing="rikai.contrib.torch.transforms.ssd.post_processing",
        registered_model_name="ssd"
    )
with mlflow.start_run():
    rikai.mlflow.pytorch.log_model(
        class_scores_extractor,
        "model_scores",
        SSDClassScoresExtractor.SCHEMA,
        pre_processing="rikai.contrib.torch.inspect.ssd.class_scores_extractor_pre_processing",
        post_processing="rikai.contrib.torch.inspect.ssd.class_scores_extractor_post_processing",
        registered_model_name="class_scores"
    )

array<struct<box:box2d, score:float, label_id:int>>


2022/01/29 06:50:19 WARNING mlflow.utils.requirements_utils: Found torch version (1.10.0+cu111) contains a local version label (+cu111). MLflow logged a pip requirement for this package as 'torch==1.10.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/01/29 06:50:19 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.11.1+cu111) contains a local version label (+cu111). MLflow logged a pip requirement for this package as 'torchvision==0.11.1' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2022/01/29 06:50:22 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.11.1+cu111) contains a local version label (+cu111). MLflow logged a pip requirement for this package as 'torchvision==0.11.1' without the loc

We then load the models from `mlflow` registry to make them available to use in Spark SQL. 

In [9]:
spark.sql("""
  CREATE OR REPLACE MODEL ssd 
  OPTIONS (batch_size=64, num_workers=4) 
  USING 'mlflow:/ssd'""")
spark.sql("""
  CREATE OR REPLACE MODEL class_scores 
  OPTIONS (batch_size=64, num_workers=4) 
  USING 'mlflow:/class_scores'""")
spark.sql("SHOW MODELS").show()

+------------+------+--------------------+--------------------+
|        name|flavor|                 uri|             options|
+------------+------+--------------------+--------------------+
|         ssd|      |         mlflow:/ssd|{"batch_size":"64...|
|class_scores|      |mlflow:/class_scores|{"batch_size":"64...|
+------------+------+--------------------+--------------------+



# Least Confidence

**Least Confidence** looks for predicted labels with the lowest degree of confidence

$$ 1 - P(y_1 | x) $$

This strategy identifies the predictions with the lowest confidence score. Using Rikai, this is a simple SQL statement that looks like the following:

In [10]:
least_confident_df = spark.sql("""
SELECT  /*+ REPARTITION(200) */ 
  image_id, image, explode(ML_PREDICT(ssd, image)) AS ssd 
FROM coco_1k
ORDER BY ssd.score ASC
LIMIT 1000
""")


We get a PySpark DataFrame once we execute this Spark SQL query and we can take a quick look at the results. Rikai is integrated with Jupyter so that it's easy to visualize your images and annotations without having to mess with low level PIL/OpenCV APIs.

In [11]:
from rikai.viz import Text

for row in least_confident_df.take(3):
    text = COCO_INSTANCE_CATEGORY_NAMES[row.ssd.label_id]
    display(row.image 
        | row.ssd.box@{"color": "yellow", "width": 3} 
        | Text(f"{text} | {row.ssd.score:.3f}", 
               (row.ssd.box.xmin + 2, row.ssd.box.ymin - 10))@{"color": "white"}
    )


Draw(style({'color': 'yellow', 'width': 3}))

Draw(style({'color': 'yellow', 'width': 3}))

Draw(style({'color': 'yellow', 'width': 3}))

# Least Margin of Confidence

**Margin of Confidence** looks for training examples with the lowest difference between most likely and second most likely labels. Intuitively, it gives insights into where the model is confused the most.

$$ P(y_1 | x) - P(y_2 | x) $$

This strategy finds predictions where the model has a hard time distinguishing between the top two classes. Here, we use Rikai's `SSDClassScoresExtractors` model to extract the classes (label) and scores for the top candidates from each bounding box:

In [12]:
margin_df = spark.sql("""
SELECT 
  image_id, image, detection,
  detection.scores[0] - detection.scores[1] as margin FROM (
    SELECT 
      image_id, 
      image, 
      explode(ML_PREDICT(class_scores, image)) AS detection 
    FROM coco_1k
) ORDER BY margin
LIMIT 10000
""")
margin_df.write.mode("overwrite").save("datasets/least_margin")
margin_df.createOrReplaceTempView("least_margin")


Again, let's inspect the query results:

In [19]:
first = margin_df.first()
label1 = COCO_INSTANCE_CATEGORY_NAMES[first.detection.label_ids[0]]
label2 = COCO_INSTANCE_CATEGORY_NAMES[first.detection.label_ids[1]]
text = f"{label1} - {label2}\n{first.margin:0.10f}"
box = first.detection.box
print(box)
(
    first.image 
    | box@{"color": "white", "width": 3} 
    | Text(text, (box.xmin + 3, box.ymin))@{"color": "white"}
)

Box2d(xmin=33.639434814453125, ymin=15.40258502960205, xmax=123.44316101074219, ymax=94.63240814208984)


Draw(style({'color': 'white', 'width': 3}))

# Entropy

**Entropy** observing the average level of uncertainty over all the labels.

$$ \frac{-\sum_{i=1}^{n}P(y_i | x)log_{2}P(y_i | x)}{log_2{n}}$$

It is a measure of the amount of information in data. The higher value the entropy is, the more similar all the values are. While margin of confidence takes into account the top two classes, entropy takes into account all predicted classes. For active learning using uncertainty sampling, we're looking for predictions with the highest entropy.

In [14]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from scipy.stats import entropy as scipyEntropy

@udf(returnType=FloatType())
def entropy(arr) -> float:
    """Calculate entropy over all scores"""
    return float(scipyEntropy(arr))

spark.udf.register("entropy", entropy)

<function __main__.entropy(arr) -> float>

In [15]:
entropy_df = spark.sql("""
SELECT image_id, image, detection, entropy(detection.scores) as entropy FROM (
    SELECT 
      image_id, 
      image,
      explode(ML_PREDICT(class_scores, image)) AS detection 
    FROM coco_1k
) ORDER BY entropy DESC
LIMIT 1000
""")
entropy_df.cache()
entropy_df.show()

+--------+--------------------+--------------------+---------+
|image_id|               image|           detection|  entropy|
+--------+--------------------+--------------------+---------+
|  127517|Image('coco/val20...|{Box2d(xmin=626.2...|4.1688676|
|   66926|Image('coco/val20...|{Box2d(xmin=0.0, ...|4.1371126|
|  127517|Image('coco/val20...|{Box2d(xmin=7.046...| 4.136606|
|  540928|Image('coco/val20...|{Box2d(xmin=232.3...| 4.135115|
|  360325|Image('coco/val20...|{Box2d(xmin=0.0, ...|4.1118207|
|  127517|Image('coco/val20...|{Box2d(xmin=20.89...|  4.10723|
|  127517|Image('coco/val20...|{Box2d(xmin=30.32...|4.1040163|
|  433134|Image('coco/val20...|{Box2d(xmin=336.8...|4.0930743|
|  320632|Image('coco/val20...|{Box2d(xmin=631.7...| 4.076382|
|   66926|Image('coco/val20...|{Box2d(xmin=592.7...|4.0742974|
|  416330|Image('coco/val20...|{Box2d(xmin=386.2...|4.0717626|
|  127517|Image('coco/val20...|{Box2d(xmin=231.8...| 4.067218|
|  407943|Image('coco/val20...|{Box2d(xmin=1.739...| 4.

In [20]:
first = entropy_df.first()
text = f"{COCO_INSTANCE_CATEGORY_NAMES[first.detection.label_ids[0]]}\n{first.entropy}"
box = first.detection.box
print(box)
(
    first.image 
    | box@{"color": "yellow", "width": 3} 
    | Text(text, (box.xmin, box.ymax))@{"color": "yellow"}
)

Box2d(xmin=626.2796630859375, ymin=249.94677734375, xmax=640.0000610351562, ymax=306.9650573730469)


Draw(style({'color': 'yellow', 'width': 3}))

# Let's do some analysis

Our analysis does not stop here. 
With these uncertainty sampling techniques, we can gain much more insights.

First, let's investigate what classes the official PyTorch SSD model confused the most:   

In [17]:
confused_label_df = spark.sql("""
SELECT 
  coco_name(first_class) as first_class,
  coco_name(second_class) as second_class,
  cnt, avg_margin
FROM (
  SELECT 
    detection.label_ids[0] as first_class,
    detection.label_ids[1] as second_class,
    count(image_id) as cnt,
    avg(margin) as avg_margin
  FROM least_margin
  GROUP BY first_class, second_class
  SORT BY cnt DESC
)
""")
confused_label_df.show()


+-----------+------------+---+--------------------+
|first_class|second_class|cnt|          avg_margin|
+-----------+------------+---+--------------------+
|      chair|      person|432|0.004260380628423...|
|     person|       chair|398|0.004080007705678367|
|      sheep|         cow|282|  0.0039944492267273|
|        cow|       sheep|239|0.003962736715130102|
|      sheep|        bird|202|0.004086090397034394|
|       bird|       sheep|194|0.004654380569676149|
|     person|        bird|182|0.004919620778329753|
|       bird|      person|179|0.004416276958339208|
|     person|         car|179|0.004098445442427137|
|        car|      person|171|0.004065256380634...|
|     remote|  cell phone|162|0.003788785741629976|
| cell phone|      remote|156|0.004183752447342...|
|     carrot|    broccoli|150|0.004193373108282685|
|   backpack|     handbag|148|0.003647504840046...|
|   broccoli|      carrot|134|0.004395675866294708|
|       bird|        kite|131|0.004566725620729777|
|    handbag

What a surprise, our model is highly likely to confuse between `Person` and `Chair`!! 

But if we dig deeper, we found that between `(remote, cell phone)` and `(sheep, cow)`, the `avg_margin`s are lower. 

We could not help to wonder: "*is it because the dataset has much more `people/chares` than `remote/cellphones` (a.k.a, imbalanced dataset)?*"

With Rikai, it is easy to verify this hyperthesis.

In [27]:
spark.sql("""
SELECT * FROM (
  SELECT 
    coco_name(detection.label_id) as label,
    count(detection.label_id) AS cnt 
  FROM (
    SELECT 
      explode(ML_PREDICT(ssd, image)) AS detection
    FROM coco_1k
  ) GROUP BY detection.label_id
)
WHERE label IN ('person', 'chair', 'remote', 'cell phone', 'sheep', 'cow')
ORDER BY cnt DESC
""").show()

+----------+---+
|     label|cnt|
+----------+---+
|    person|949|
|     chair| 59|
|     sheep| 47|
|       cow| 37|
|cell phone| 12|
|    remote|  3|
+----------+---+



Great, our hyperthesis has been proved, `people` has 60 times more instances than `cell phone` and `remote` combined!